# Load libraries

In [13]:
import os
import h5py
import pickle
import numpy as np


In [14]:
subjNums = [ 'sub-3', 'sub-4', 'sub-5', 'sub-6', 'sub-7', 'sub-8', 'sub-10', 
         'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20',
           'sub-21', 'sub-22', 'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-28', 'sub-29', 'sub-30',
          'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38', 'sub-39', 'sub-40']

In [15]:
actual_data_dir = '/projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/'
actual_suffix = '_actualBetas.pkl'   

output_dir = '/projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwiseH5/'


# convert pickle to h5 

In [16]:
def load_and_print_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
        return data

allRuns = ['test1_901', 'test2_1101', 'test3_1301', 'test4_1501', 
           'test5_1701', 'test6_1901', 'test7_2101', 'test8_2301']

In [17]:
def preprocess(subj):
    
    output_path = os.path.join(output_dir, subj + '_actualBetas.h5')

    with h5py.File(output_path, 'w') as h5f:
        for run in allRuns:
            run_path = os.path.join(actual_data_dir, subj, subj + '_' + run + actual_suffix)

            try:
                run_data = load_and_print_pickle(run_path)
            except FileNotFoundError:
                print(f'File not found: {run_path}')
                continue

            run_group = h5f.create_group(run)

            for miniblock in run_data:
                mb_num = miniblock['miniblock']
                mb_group = run_group.create_group(f'miniblock_{mb_num}')
                
                trials = miniblock['trials']
                conditions = [trial['condition'] for trial in trials]
                betas = np.stack([trial['betas'] for trial in trials])
                
                mb_group.create_dataset('condition', data=np.array(conditions, dtype='S'))
                mb_group.create_dataset('betas', data=betas)

                # Handle motor_response if exists in any trial
                if any('motor_response' in trial for trial in trials):
                    motor_responses = [trial.get('motor_response', '') for trial in trials]
                    mb_group.create_dataset('motor_response', data=np.array(motor_responses, dtype='S'))

                # Handle Stimuli if exists in any trial
                if any('Stimuli' in trial for trial in trials):
                    stimuli_vals = [trial.get('Stimuli', np.nan) for trial in trials]
                    mb_group.create_dataset('Stimuli', data=np.array(stimuli_vals))


In [18]:
for subj in subjNums:
    preprocess(subj)

File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-16/sub-16_test7_2101_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-21/sub-21_test4_1501_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-29/sub-29_test5_1701_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-31/sub-31_test7_2101_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-32/sub-32_test2_1101_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-35/sub-35_test3_1301_actualBetas.pkl
File not found: /projects/f_mc1689_1/MeiranNext/data/results/ArunResults/ActualBetasParcelwise/sub-38/sub-38_test5_1701_actualBetas.pkl
